In [ ]:
# for n in [number of time steps, wind change frequency]
    # solve IP for u_0 (for t_n = t_0, use prior on u_0)
    # solve OC for c(t) from t_n->t_n+1
    # gather data along c(t) from t_n->t_n+1

  

In [ ]:
import dolfin as dl
import ufl
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sys
import os
sys.path.append( os.environ.get('HIPPYLIB_BASE_DIR', "../../../") )
sys.path.append("../../../")
sys.path.append("../")
from hippylib import *
sys.path.append( os.environ.get('HIPPYLIB_BASE_DIR', "..") + "/applications/ad_diff/" )
from applications.ad_diff.model_ad_diff_bwd import TimeDependentAD, SpaceTimePointwiseStateObservation
from fourier import *; from bcs import *
import logging
logging.getLogger('FFC').setLevel(logging.WARNING)
logging.getLogger('UFL').setLevel(logging.WARNING)
dl.set_log_active(False)

In [ ]:
nx, ny = 60, 120
mesh = dl.UnitSquareMesh(nx, ny)

for _ in range(2):  
    markers = dl.MeshFunction("bool", mesh, mesh.topology().dim())
    markers.set_all(False)

    for cell in dl.cells(mesh):
        y = cell.midpoint().y()
        if y < 0.06 or y > 0.94:     
            markers[cell] = True

    mesh = dl.refine(mesh, markers)

wind_velocity = computeVelocityField(mesh)
Vh = dl.FunctionSpace(mesh, "Lagrange", 1)
print("Number of dofs: {0}".format(Vh.dim()))

In [ ]:
ic_expr = dl.Expression(
    'std::min(0.5, std::exp(-100*(std::pow(x[0]-0.35,2) + std::pow(x[1]-0.7,2))))',
    element=Vh.ufl_element()
)
true_initial_condition = dl.interpolate(ic_expr, Vh).vector()

gamma = 1.
delta = 8.
prior = BiLaplacianPrior(Vh, gamma, delta, robin_bc=True)
prior.mean = dl.interpolate(dl.Constant(0.25), Vh).vector()

t_init = 0.
t_final = 4.
t_1 = 1.
dt = .1
observation_dt = .2

simulation_times = np.arange(t_init, t_final + .5*dt, dt)
observation_times = np.arange(t_1, t_final + .5*dt, observation_dt)

Ty = (t_1, t_final)
Nf = 3
omegas = fourier_frequencies(Ty, Nf)
coeffs_v = np.array([
    [ 0.05,  0.00],
    [ 0.02,  0.01],
    [ 0.01, -0.01],
], dtype=float)

coeffs_w = np.array([
    [ 0.00,  0.10],
    [ 0.00,  0.05],
    [ 0.00, -0.02],
], dtype=float)

x0 = np.array([0.35, 0.70], dtype=float)  
theta0 = 0.0                              

targets = integrate_unicycle_path(
    observation_times,
    x0=x0,
    theta0=theta0,
    omegas=omegas,
    coeffs_v=coeffs_v,
    coeffs_w=coeffs_w,
    v0=.15,
    w0=.10,
    eps=1e-3,
    substeps=20
)

print("Number of observation points:", targets.shape[0])
assert targets.shape[0] == len(observation_times)

misfit = SpaceTimePointwiseStateObservation(Vh, observation_times, targets)



problem = TimeDependentAD(mesh, [Vh, Vh, Vh], prior, misfit, simulation_times, wind_velocity, True)

objs = [
    dl.Function(Vh, true_initial_condition),
    dl.Function(Vh, prior.mean)
]
mytitles = ["True Initial Condition", "Prior mean"]
nb.multi1_plot(objs, mytitles)
plt.show()